In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

#print(df.head(10))
print(df.shape)
print(df.info())

(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


## 🔱주요 컬럼:

PassengerId: 승객 ID

Survived: 생존 여부 (0 = 사망, 1 = 생존)

Pclass: 객실 등급 (1, 2, 3)

Name: 이름 (예: Braund, Mr. Owen Harris)

Sex: 성별 (male, female)

Age: 나이 (결측값 다수 포함)

SibSp: 함께 탑승한 형제자매/배우자 수

Parch: 함께 탑승한 부모/자녀 수

Ticket: 티켓 번호

Fare: 운임

Cabin: 선실 번호 (예: C85, 결측값 매우 많음)

Embarked: 탑승 항구 (S, C, Q)

💻 문제 1. (결측값 채우기 - 고급) Age 컬럼의 결측값(NaN)이 많습니다. 이 결측값을 **Pclass (객실 등급) 별 Age의 중앙값(median)**으로 채워 넣으려고 합니다. (예: 1등석(Pclass=1) 승객의 Age 결측값은 1등석 승객 전체의 Age 중앙값으로 채움) 결측값을 모두 채운 후, Age 컬럼의 전체 평균값을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 계산)

In [2]:
# 1번 문제(transform)
# df.groupby('<group_col>')['<calc_col>'].transform('<function>')

age_fill_values = df.groupby('Pclass')['Age'].transform('median')
df['Age'] = df['Age'].fillna(age_fill_values)

result = df['Age'].mean()
print(round(result, 2))
print(age_fill_values)


29.07
0      24.0
1      37.0
2      24.0
3      37.0
4      24.0
       ... 
886    29.0
887    37.0
888    24.0
889    37.0
890    24.0
Name: Age, Length: 891, dtype: float64


In [3]:
# 1번 문제(loc)
# df.loc[<row_condition>, <column_name>] = new_value

medians = df.groupby('Pclass')['Age'].median()
# medians[1] = 1등석 중앙값
# medians[2] = 2등석 중앙값
# medians[3] = 3등석 중앙값

df.loc[(df['Pclass'] == 1) & (df['Age'].isna()), 'Age'] = medians[1]
df.loc[(df['Pclass'] == 2) & (df['Age'].isna()), 'Age'] = medians[2]
df.loc[(df['Pclass'] == 3) & (df['Age'].isna()), 'Age'] = medians[3]

result = df['Age'].mean()
print(round(result, 2))

print(medians[1])

29.07
37.0


🚀 요약: 그래서 언제 뭘 쓸까?
loc (수술): "딱 이 칸, 저 칸만 골라서" 값을 바꾸고 싶을 때.

df.loc[<조건>, <바꿀컬럼>] = <새값>

transform (공정): "그룹별 계산 값을" "원본 모든 행에" 매핑해서 사용하고 싶을 때.

df.groupby(...).transform(...) (주로 fillna와 찰떡궁합입니다.)

💻 문제 2. (문자열 추출 - Regex) Name 컬럼에는 승객의 호칭(Title)이 포함되어 있습니다. (예: Mr., Mrs., Miss., Master. 등) Name 컬럼에서 **정규표현식(Regex)**을 사용하여 이 호칭([A-Za-z]+\.)을 추출하여 'Title'이라는 새 컬럼을 만드시오. 그 후, 'Title'별 인원수를 집계할 때, 10명 이상의 인원을 가진 'Title' 그룹들의 Age 평균을 구하시오. (단, 소수점 셋째 자리에서 반올림하여 둘째 자리까지 계산)

In [4]:
# 2번 문제
df['Title'] = df['Name'].str.extract('([A-Za-z]+)\.')
title_counts = df['Title'].value_counts()

target_titles = title_counts[title_counts >= 10].index
target_df = df[df['Title'].isin(target_titles)]
result = target_df['Age'].mean()
print(round(result, 2))

28.66


💻 문제 3. (결측값 + 문자열 처리 - 복합) Cabin 컬럼은 결측값이 매우 많습니다.

Cabin 컬럼의 결측값은 'Unknown'으로 채웁니다.

Cabin 컬럼이 결측값이 아닌 경우, 값의 **첫 번째 알파벳(Deck)**만 추출합니다. (예: C85 -> C, G6 -> G)

Unknown으로 채워진 데이터는 Deck를 'U'로 변경합니다. (힌트: fillna('U') 후 .str[0])

이렇게 생성된 Deck 컬럼(A, B, C... U)별 Survived (생존율) 평균을 계산할 때, 생존율이 가장 높았던 Deck의 이름을 출력하시오.

In [5]:
# 3번 문제
df['Cabin'] = df['Cabin'].fillna('Unknown')
#print(df['Cabin'].head(10))

q1 = df[['Cabin', 'Survived']]

deck = q1['Cabin'].str[0]
q1['Cabin'] = deck

survived_rates = q1.groupby('Cabin')['Survived'].mean()
print(survived_rates)


Cabin
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
T    0.000000
U    0.299854
Name: Survived, dtype: float64


C:\Users\sangh\AppData\Local\Temp\ipykernel_15632\403286857.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q1['Cabin'] = deck


In [6]:
# 3번 문제 완성 코드
df['Deck'] = df['Cabin'].fillna('U').str[0]

deck_survival_rates = df.groupby('Deck')['Survived'].mean()

print(deck_survival_rates)

Deck
A    0.466667
B    0.744681
C    0.593220
D    0.757576
E    0.750000
F    0.615385
G    0.500000
T    0.000000
U    0.299854
Name: Survived, dtype: float64


🚀 핵심: Survived 컬럼 (0과 1로만 구성)의 .mean()을 구하면, 그게 바로 1의 비율, 즉 생존율이 된다. (예: [1, 0, 1]의 평균은 0.66 -> 66%)

💻 문제 4. (Feature 생성 및 Categorization) SibSp (형제/배우자)와 Parch (부모/자녀) 컬럼을 합쳐, 본인을 포함한 'FamilySize' (가족 총원 = SibSp + Parch + 1) 컬럼을 생성합니다. 이 'FamilySize'를 기준으로 다음과 같이 'FamilyGroup'을 분류합니다.

1명: 'Alone'

2~4명: 'Small'

5명 이상: 'Large' 

'FamilyGroup'별 Survived (생존율) 평균을 계산하고, 생존율이 가장 높았던 그룹의 이름을 출력하시오.

In [7]:
# 4번 문제
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

df['FamilyGroup'] = 'Unknown'

df.loc[df['FamilySize']==1, 'FamilyGroup'] = 'Alone'
df.loc[(df['FamilySize'] >= 2) & (df['FamilySize'] <= 4), 'FamilyGroup'] = 'Small'
df.loc[df['FamilySize']>=5, 'FamilyGroup'] = 'Large'

result = df.groupby('FamilyGroup')['Survived'].mean()
print(result)





FamilyGroup
Alone    0.303538
Large    0.161290
Small    0.578767
Name: Survived, dtype: float64


💻 문제 5. (복합 조건 apply 및 Feature 생성) Age 컬럼 (결측값이 10번 문제 방식으로 채워졌다고 가정)과 Sex 컬럼을 조합하여 'AgeSexGroup'을 만듭니다.

'male'이면서 18세 미만: 'Boy'

'female'이면서 18세 미만: 'Girl'

'male'이면서 18세 이상: 'Man'

'female'이면서 18세 이상: 'Woman' 

가장 많은 인원수를 차지하는 'AgeSexGroup'의 이름을 출력하시오.

In [8]:
# 결측값 처리
age_fill_values = df.groupby('Pclass')['Age'].transform('median')
df['Age'] = df['Age'].fillna(age_fill_values)

# 5번 문제
df['AgeSexGroup'] = 'Unknown'

df.loc[(df['Sex']=='male') & (df['Age'] < 18), 'AgeSexGroup'] = 'Boy'
df.loc[(df['Sex']=='female') & (df['Age'] < 18), 'AgeSexGroup'] = 'Girl'
df.loc[(df['Sex']=='male') & (df['Age'] >= 18), 'AgeSexGroup'] = 'Man'
df.loc[(df['Sex']=='female') & (df['Age'] >= 18), 'AgeSexGroup'] = 'Woman'

result = df['AgeSexGroup'].value_counts()
print(result)




AgeSexGroup
Man      519
Woman    259
Boy       58
Girl      55
Name: count, dtype: int64


💻 문제 6. (Pivot Table 및 .loc) (10번 문제의 Age 결측값 처리, 12번 문제의 Deck('U' 포함) 처리가 완료되었다고 가정합니다.) pivot_table을 사용하여 'Deck'을 index로, 'Pclass'를 columns로, Age의 평균을 values로 하는 피벗 테이블을 생성합니다. 생성된 피벗 테이블에서, 'U' (Unknown) 데크의 3등석(Pclass=3) 승객 평균 나이를 소수점 셋째 자리에서 반올림하여 둘째 자리까지 출력하시오.

In [9]:
# 결측값 처리
df['Deck'] = df['Cabin'].fillna('U').str[0]

# 6번 문제
pivot_df = df.pivot_table(index='Deck', columns='Pclass', values='Age', aggfunc='mean')

print(round(pivot_df.loc['U', 3], 2))

24.91


### 💻 제2유형: 데이터 모형 구축
타이타닉 생존자 예측
[훈련 데이터(train.csv)로 학습한 모델을 테스트 데이터(test.csv)에 적용하여 예측한 결과를 제출하시오.

Target: Survived (생존 여부, 이진 분류, {0, 1})

제출형식: result.csv

ID, pred 칼럼만 존재해야 합니다.

(주의: test.csv의 PassengerId를 ID 칼럼으로 사용해야 함)

평가지표: Accuracy (정확도) ]

In [10]:
train_url = "https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv"
test_url = "https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv"

train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

# 결측치 처리
#print(test.info())
# trian: age, cabin, embarked
# test: age, cabin

age_medians_by_pclass = train.groupby('Pclass')['Age'].median().to_dict()
train['Age'] = train['Age'].fillna(train.groupby('Pclass')['Age'].transform('median'))
test['Age'] = test['Age'].fillna(test['Pclass'].map(age_medians_by_pclass))

train['Cabin'] = train['Cabin'].fillna('Unknown')
test['Cabin'] = test['Cabin'].fillna('Unknown')

fill_embarked = train['Embarked'].mode()[0]
train['Embarked'] = train['Embarked'].fillna(fill_embarked)

fare_mean_train = train['Fare'].mean()
test['Fare'] = test['Fare'].fillna(fare_mean_train)

# 인코딩
#print(test.info())
# train 인코딩: Sex, Cabin
# train 제거: PassengerId, Ticket, Embarked

X = train.drop(['Name', 'Survived', 'PassengerId', 'Ticket', 'Embarked'], axis=1)
y = train['Survived']

X_test = test.drop(['Name', 'PassengerId', 'Ticket', 'Embarked'], axis=1)

train_len = len(X)
combined_df = pd.concat([X, X_test], axis=0)

cols = ['Sex', 'Cabin']
combined_df_dummies = pd.get_dummies(combined_df, columns=cols)

X_final = combined_df_dummies.iloc[:train_len]
X_test_final = combined_df_dummies.iloc[train_len:]

# 값 불일치
#X_train = pd.get_dummies(X_train, columns=cols)
#X_test = pd.get_dummies(X_test, columns=cols)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

x_train, x_val, y_train, y_val = train_test_split(X_final, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)
pred = model.predict(x_val)

auc_score = roc_auc_score(y_val, pred)
print(round(auc_score, 3))

# 전체 데이터 재학습
model = RandomForestClassifier(random_state=42)
# 한번 더 선언? -> 훈련 데이터 100%로 재학습
model.fit(X_final, y)
final_pred = model.predict(X_test_final)

result = pd.DataFrame({
    'ID': test['PassengerId'],
    'pred': final_pred
})
result.to_csv('result.csv', index=False)
print(result)


0.771
       ID  pred
0     892     0
1     893     0
2     894     1
3     895     1
4     896     0
..    ...   ...
413  1305     0
414  1306     1
415  1307     0
416  1308     0
417  1309     0

[418 rows x 2 columns]


### 💻 제3유형: 데이터 모형 평가

① 1등석과 3등석, 두 집단의 Age 분산에 차이가 있는지를 알아보기 위해 **F-검정(F-test)**을 수행할 때 검정통계량의 값을 구하여라. (단, 분산 계산 시 1등석의 분산을 분자, 3등석의 분산을 분모로 하여 계산한다.)

【제출 형식】 반올림하여 소수 셋째 자리까지 작성

② 1번의 F-검정 p-값을 구하여라. 이 p-값을 바탕으로 두 집단의 등분산성 가정을 할 수 있는지 귀무가설과 대립가설에 근거하여 (채점용 답안이 아닌) 자유롭게 서술하시오. (유의수준: 0.05)

【제출 형식】 (1) p-값 (반올림하여 소수 셋째 자리까지 작성), (2) 서술형 답안

③ ②번의 결과와 관계없이, 등분산성(equal variance)을 가정하고 두 집단의 Age에 유의미한 차이가 있는지 **독립표본 t-검정(Independent t-test)**을 수행하고 p-값을 구하여라.

【제출 형식】 반올림하여 소수 셋째 자리까지 작성

In [11]:
#--- 1. 데이터 준비 (필수 선행) ---
train_url = "https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv"
df = pd.read_csv(train_url)

# Age 결측값 채우기
age_fill_values = df.groupby('Pclass')['Age'].transform('median')
df['Age'] = df['Age'].fillna(age_fill_values)

group1 = df[df['Pclass']==1]['Age']
group3 = df[df['Pclass']==3]['Age']

from scipy.stats import f, ttest_ind
import numpy as np
var1 = np.var(group1, ddof=1)
var3 = np.var(group3, ddof=1)
f_stat = var1 / var3
print(round(f_stat, 3))

dfn = len(group1) - 1
dfd = len(group3) - 1
p_value = 1 - f.cdf(f_stat, dfn=dfn, dfd=dfd)
print(round(p_value, 3))

t_stat, p_value_ttest = ttest_ind(group1, group3, equal_var=True)
print(round(p_value_ttest, 3))


1.669
0.0
0.0


알겠습니다. 방금 정리한 통계 개념들을 타이타닉 데이터셋에 적용해보는 실전 문제 3세트를 만들었습니다.

-----

### (체험) 제3유형 (풀이용)

**유형:** 통계 검정 (Python)

제공된 데이터(train.csv)는 타이타닉 승객 명단입니다. 통계 검정을 수행하기 전, `Age`는 10번 문제와 동일하게 **Pclass별 중앙값으로 결측치를 채웠고**, `Embarked`는 **최빈값('S')으로 결측치를 채웠다**고 가정합니다.

> **제공 데이터 (`train.csv`)**
>
>   * **Survived:** 생존 여부 (0: 사망, 1: 생존)
>   * **Pclass:** 객실 등급 (1, 2, 3)
>   * **Age:** 나이 (결측값 처리 완료)
>   * **Fare:** 요금
>   * **Embarked:** 탑승 항구 (S, C, Q / 결측값 처리 완료)

-----

### 📜 문제 1 (t-검정)

생존 여부(`Survived`)에 따라 승객의 평균 나이(`Age`)에 유의미한 차이가 있는지 검정하려고 합니다.

**①** 생존자 그룹(`Survived`=1)과 사망자 그룹(`Survived`=0)의 **등분산성을 Levene 검정**으로 확인하고자 한다. 이때의 **p-값**을 구하시오.

> **【제출 형식】** 반올림하여 소수 셋째 자리까지 작성

**②** 1번 문제의 p-값과 관계없이, 두 집단이 **등분산성(equal\_var=True)을 만족한다고 가정**하고 **독립표본 t-검정**을 수행하였다. 이때의 **p-값**을 구하시오.

> **【제출 형식】** 반올림하여 소수 셋째 자리까지 작성

### 📜 문제 2 (카이제곱 검정)

객실 등급(`Pclass`)과 생존 여부(`Survived`)가 서로 **독립적인지(관련이 없는지)** 검정하려고 합니다.

**①** `pd.crosstab`을 이용해 교차표를 생성한 뒤, **카이제곱 독립성 검정**을 수행하시오. 이때의 \*\*검정통계량($\chi^2$)\*\*을 구하시오.

> **【제출 형식】** 반올림하여 소수 셋째 자리까지 작성

**②** 1번 문제에서 수행한 카이제곱 검정의 **p-값**을 구하시오.

> **【제출 형식】** 반올림하여 소수 넷째 자리까지 작성 (보통 p-값은 더 정밀하게 봅니다)

### 📜 문제 3 (ANOVA + 사후검정)

탑승 항구(`Embarked`)별로 승객들이 지불한 요금(`Fare`)의 평균에 유의미한 차이가 있는지 검정하려고 합니다.

**①** 세 항구(S, C, Q) 간의 평균 `Fare` 차이를 검정하기 위해 \*\*일원분산분석(ANOVA)\*\*을 수행하였다. 이때의 **F-통계량**을 구하시오.

> **【제출 형식】** 반올림하여 소수 셋째 자리까지 작성

**②** 1번 문제에서 수행한 ANOVA 검정의 **p-값**을 구하시오.

> **【제출 형식】** 반올림하여 소수 넷째 자리까지 작성

**③** 2번 문제의 p-값이 0.05보다 작아 귀무가설이 기각되었다. **Tukey HSD 사후검정**을 수행하여 'S' 항구와 'C' 항구 간의 `p-adj` (조정된 p-값)을 구하시오.

> **【제출 형식】** 반올림하여 소수 셋째 자리까지 작성

In [12]:
group1 = df[df['Survived']==1]['Age']
group0 = df[df['Survived']==0]['Age']

from scipy.stats import levene, ttest_ind

stat, p_levene = levene(group1, group0)
print(round(p_levene, 3))

t_stat, p_value = ttest_ind(group1, group0, equal_var=True)
print(round(p_value, 3))

from scipy.stats import chi2_contingency
ct = pd.crosstab(df['Pclass'], df['Survived'])
chi2, p, dof, expected = chi2_contingency(ct)
print(round(chi2, 3))
print(round(p, 4))

from scipy import stats
group_S = df[df['Embarked']=='S']['Fare']
group_C = df[df['Embarked']=='C']['Fare']
group_Q = df[df['Embarked']=='Q']['Fare']
f_stat, p_anova = stats.f_oneway(group_S, group_C, group_Q)
print(round(f_stat, 3))
print(round(p_anova, 4))

# 분산분석에 대해 더 자세히 설명, 어떨때 scipy / statsmodels 쓰는지, Tukey에 대해서도 자세히 설명. 마크다운으로.


0.036
0.159
102.889
0.0
38.14
0.0
